In [39]:
import torch
import cv2
import os

# Load YOLOv5s model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Set image directory
image_dir = '../images/semaforos/train_images'

# Create output directory
output_dir = '../images/semaforos/simples'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop through images in directory
for filename in os.listdir(image_dir):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Load image
        img_path = os.path.join(image_dir, filename)
        img = cv2.imread(img_path)

        # Detect objects
        results = model(img)

        # Filter results to stoplights only
        stoplight_results = results.pred[0][results.pred[0][:, 5] == 9]

        # Draw bounding boxes around stoplights and save cropped images
        for result in stoplight_results:
            x1, y1, x2, y2 = result[:4].int()
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,0), 2)
            stoplight_img = img[y1:y2, x1:x2]
            cv2.imwrite(os.path.join(output_dir, filename), stoplight_img)

cv2.destroyAllWindows()


Using cache found in C:\Users\Enrique/.cache\torch\hub\ultralytics_yolov5_master
requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...

requirements: 1 package updated per C:\Users\Enrique\.cache\torch\hub\ultralytics_yolov5_master\requirements.txt
requirements:  Restart runtime or rerun command for updates to take effect

YOLOv5  2023-3-9 Python-3.10.4 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [45]:
import torch
import cv2
import os

# Load YOLOv5s model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Set image directory
image_dir = '../images/semaforos/simples'

# Create output directories
output_dir_red = '../images/semaforos/simples/red'
output_dir_yellow = '../images/semaforos/simples/yellow'
output_dir_green = '../images/semaforos/simples/green'
output_dir_other = '../images/semaforos/simples/otros'


for output_dir in [output_dir_red, output_dir_yellow, output_dir_green,output_dir_other]:
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

# Define color ranges for each traffic light color in the HSV color space
color_ranges = {
    'red': [(0, 50, 50), (10, 255, 255)],
    'yellow': [(20, 50, 50), (30, 255, 255)],
    'green': [(60, 50, 50), (80, 255, 255)]
}


# Loop through images in directory
for filename in os.listdir(image_dir):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Load image
        img_path = os.path.join(image_dir, filename)
        img = cv2.imread(img_path)

        # Convert image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Apply HoughCircles to detect circles in the image
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=40, minRadius=20, maxRadius=50)

        # If circles were found, classify the image based on the color of the largest circle
        if circles is not None:
            # Get largest circle
            circles = np.round(circles[0, :]).astype("int")
            largest_circle = max(circles, key=lambda circle: circle[2])

            # Get bounding box coordinates
            x, y, r = largest_circle
            x1, y1 = x - r, y - r
            x2, y2 = x + r, y + r

            # Crop stoplight image
            stoplight_img = img[y1:y2, x1:x2]

            # Check if stoplight image is not empty
            if stoplight_img is not None and stoplight_img.size > 0:
                # Convert stoplight image to HSV color space
                hsv_stoplight = cv2.cvtColor(stoplight_img, cv2.COLOR_BGR2HSV)

                # Threshold the stoplight image based on color ranges to separate colors
                color_match = False
                for color, ranges in color_ranges.items():
                    mask = cv2.inRange(hsv_stoplight, ranges[0], ranges[1])
                    if cv2.countNonZero(mask) > 0:
                        # If mask is not all black, save image to corresponding color directory
                        output_dir = output_dir_red if color == 'red' else output_dir_yellow if color == 'yellow' else output_dir_green
                        output_path = os.path.join(output_dir, filename)
                        cv2.imwrite(output_path, img)
                        color_match = True
                        break

                # If none of the color masks matched, move image to "other" directory
                if not color_match:
                    output_path = os.path.join(output_dir_other, filename)
                    os.rename(img_path, output_path)
        else:
            # If no circles were found, move image to "other" directory
            output_path = os.path.join(output_dir_other, filename)
            os.rename(img_path, output_path)

cv2.destroyAllWindows()


Using cache found in C:\Users\Enrique/.cache\torch\hub\ultralytics_yolov5_master
requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...

requirements: 1 package updated per C:\Users\Enrique\.cache\torch\hub\ultralytics_yolov5_master\requirements.txt
requirements:  Restart runtime or rerun command for updates to take effect

YOLOv5  2023-3-9 Python-3.10.4 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


### Aqui detectamos imagenes con circulos rojo y guardamos en carpeta rojo

In [71]:
import cv2
import numpy as np
import os

# Set paths
input_dir = "../images/semaforos/simples"
output_dir = "../images/semaforos/red"

# Loop through images in input directory
for filename in os.listdir(input_dir):
    # Load image
    img = cv2.imread(os.path.join(input_dir, filename))

    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to smooth the image and reduce noise
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Detect circles using HoughCircles function
    circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=0, maxRadius=0)

    # If no circles are detected, print "False"
    if circles is None:
        print(f"{filename}: False")
    else:
        # Convert the (x, y) coordinates and radius of the circles to integers
        circles = np.round(circles[0, :]).astype("int")

        # Loop over the detected circles
        for (x, y, r) in circles:
            # Check if the circle is red
            b, g, r = img[y, x]
            if r > 2 * g and r > 2 * b:
                # Save the image to output directory
                output_path = os.path.join(output_dir, filename)
                cv2.imwrite(output_path, img)
                print(f"{filename}: True")
                break
        else:
            print(f"{filename}: False")


00004.jpg: False
00015.jpg: False
00025.jpg: True
00028.jpg: False
00029.jpg: False
00039.jpg: False
00045.jpg: False
00046.jpg: False
00049.jpg: True
00063.jpg: False
00065.jpg: False
00068.jpg: False
00072.jpg: False
00073.jpg: False
00087.jpg: False
00088.jpg: False
00090.jpg: False
00093.jpg: False
00098.jpg: False
00123.jpg: False
00125.jpg: True
00127.jpg: True
00128.jpg: False
00129.jpg: False
00130.jpg: False
00140.jpg: False
00143.jpg: False
00148.jpg: False
00152.jpg: False
00159.jpg: False
00165.jpg: False
00168.jpg: False
00180.jpg: False
00189.jpg: False
00196.jpg: False
00199.jpg: False
00208.jpg: True
00223.jpg: False
00226.jpg: False
00229.jpg: False
00232.jpg: False
00238.jpg: False
00239.jpg: False
00240.jpg: False
00242.jpg: True
00247.jpg: True
00269.jpg: True
00295.jpg: False
00304.jpg: False
00306.jpg: True
00324.jpg: False
00325.jpg: False
00326.jpg: False
00336.jpg: False
00343.jpg: True
00354.jpg: False
00374.jpg: False
00382.jpg: True
00383.jpg: True
00389.jpg

## Aqui detectamos buscando verdes

In [76]:
# Set paths
input_dir = "../images/semaforos/simples"
output_dir = "../images/semaforos/green"

# Loop through images in input directory
for filename in os.listdir(input_dir):
    # Load image
    img = cv2.imread(os.path.join(input_dir, filename))

    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to smooth the image and reduce noise
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Detect circles using HoughCircles function
    circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=0, maxRadius=0)

    # If no circles are detected, print "False"
    if circles is None:
        print(f"{filename}: False")
    else:
        # Convert the (x, y) coordinates and radius of the circles to integers
        circles = np.round(circles[0, :]).astype("int")

        # Loop over the detected circles
        for (x, y, r) in circles:
            # Check if the circle is red
            b, g, r = img[y, x]
            if g > 2 * r and g > 2 * b:
                # Save the image to output directory
                output_path = os.path.join(output_dir, filename)
                cv2.imwrite(output_path, img)
                print(f"{filename}: True")
                break
        else:
            print(f"{filename}: False")

00004.jpg: False
00015.jpg: False
00025.jpg: False
00028.jpg: False
00029.jpg: False
00039.jpg: False
00045.jpg: False
00046.jpg: False
00049.jpg: False
00063.jpg: False
00065.jpg: False
00068.jpg: False
00072.jpg: False
00073.jpg: False
00087.jpg: False
00088.jpg: False
00090.jpg: False
00093.jpg: False
00098.jpg: False
00123.jpg: False
00125.jpg: False
00127.jpg: False
00128.jpg: False
00129.jpg: False
00130.jpg: False
00140.jpg: False
00143.jpg: False
00148.jpg: False
00152.jpg: False
00159.jpg: False
00165.jpg: False
00168.jpg: False
00180.jpg: False
00189.jpg: False
00196.jpg: False
00199.jpg: False
00208.jpg: False
00223.jpg: False
00226.jpg: False
00229.jpg: False
00232.jpg: False
00238.jpg: False
00239.jpg: False
00240.jpg: False
00242.jpg: False
00247.jpg: False
00269.jpg: False
00295.jpg: False
00304.jpg: False
00306.jpg: False
00324.jpg: False
00325.jpg: False
00326.jpg: False
00336.jpg: False
00343.jpg: False
00354.jpg: False
00374.jpg: False
00382.jpg: False
00383.jpg: Fal